In [6]:
# Install a conda package in the current Jupyter kernel
import sys
!conda install --yes --prefix {sys.prefix} wget

Solving environment: done

# All requested packages already installed.



# Load the data from the file into a dataframe 

In [17]:
# The code was removed by Watson Studio for sharing.

In [18]:
df_prem_customers.head()

,sr_number,subject,description,ea_number,recordtype_name,sr_createddate,sr_closeddate,gss_entitlement_code,entitlement_desc,gss_initial_priority,severity,Customer_Level
0,19082138602,When i try to add an async workflow object the...,When i try to add an async workflow object the...,112471845,GSS Technical Support,2019-02-13,2019-02-26,PLT,Production Support Agreement,2,2,PRODUCTION
1,19015342007,System Management,Issue : Unable to power on the VM it shows fi...,114725031,GSS Technical Support,2019-07-19,2019-07-20,PLT,Production Support Agreement,3,2,PRODUCTION
2,19048599101,The PowerOnVM_Task submitted to VMware vCenter...,The PowerOnVM_Task submitted to VMware vCenter...,113821691,GSS Technical Support,2019-01-15,2019-01-15,PLT,Production Support Agreement,3,3,PRODUCTION
3,19074923902,Need to configure LogInsight Version (4.7.0-99...,Need to configure LogInsight Version (4.7.0-99...,113310139,GSS Technical Support,2019-02-06,2019-02-15,MCS,Mission Critical Support,4,4,PREMIER
4,19175312904,Storage,Unable to perform snapshot of VM 1. Is...,115883764,GSS Technical Support,2019-04-24,2019-04-26,HCS,Healthcare Critical Support,1,2,PREMIER


In [19]:
df_prem_customers.columns

Index(['sr_number', 'subject', 'description', 'ea_number', 'recordtype_name',
       'sr_createddate', 'sr_closeddate', 'gss_entitlement_code',
       'entitlement_desc', 'gss_initial_priority', 'severity',
       'Customer_Level'],
      dtype='object')

In [20]:
df_prem_customers['severity'].value_counts()

2    30702
3    17829
4     4969
1     1812
5     1404
Name: severity, dtype: int64

# Filter out Sev 5 SRs

In [22]:
df_base=df_prem_customers.loc[df_prem_customers['severity']!=5,]

In [23]:
df_base['description'].replace('', np.nan, inplace=True)
df_base['description'].dropna(inplace=True)
df_base['description']=df_base['description'].apply(lambda x: str(x))

In [24]:
import xgboost as xgb
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import getpass
import re

In [25]:
X = df_base['description']
y = df_base['severity']
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.25,
                                                    random_state = 0)

In [27]:
nltk.download('stopwords')

True

In [28]:
stopwords=nltk.corpus.stopwords.words('english')
tf = TfidfVectorizer(ngram_range=(1,3), max_features=20000, stop_words=stopwords,min_df=5)
train_feat = tf.fit_transform(X_train)
test_feat = tf.transform(X_test)

In [29]:
xgb_model = xgb.XGBClassifier(objective="multi:softprob", random_state=42)
xgb_model.fit(train_feat, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='multi:softprob', random_state=42,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1)

In [31]:
y_pred=xgb_model.predict(test_feat)

In [32]:
print('Training accuracy',accuracy_score(xgb_model.predict(train_feat),y_train))
print('Test accuracy',accuracy_score(xgb_model.predict(test_feat),y_test))

Training accuracy 0.5994118214251277
Test accuracy 0.5868527625108475


In [33]:
y_class=y.astype(str)

In [34]:
print(metrics.classification_report(y_test, y_pred, target_names= y_class.unique()))

              precision    recall  f1-score   support

           2       0.00      0.00      0.00       458
           3       0.58      0.98      0.73      7669
           4       0.70      0.12      0.20      4480
           1       0.62      0.04      0.07      1221

   micro avg       0.59      0.59      0.59     13828
   macro avg       0.47      0.29      0.25     13828
weighted avg       0.60      0.59      0.48     13828



In [35]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient